In [8]:
import os, re
from scripts.tools.issuetracker import IssuetrackerAPI
from datetime import date

In [9]:
toplevel_match = re.compile(r"^\>?([\+\*]{2})(FRS\d+)([\+\*]{2})(.*)$").match
subitem_match = re.compile(r"^\>?([\+\*]+)\s\*(FRS)([\d\.]+)\:\*(.+)$").match
header_match = re.compile(r"^\>?([\+\*]{2})([\w]+[^\:]+)(\:)?([\+\*]{2})(\:)?(.*)$").match

In [16]:
cache = None
api = None
sync_date = None
def _download():
    global cache, api, sync_date
    if api is None:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    cache = api.download_issues(project_id="pbssoftware")
    sync_date = date.today()
    return cache.copy()

In [196]:

    
def download(force=False):
    global cache, sync_date
    reason = None
    if cache is None:
        reason = "cache is None"
    elif force:
        reason = "forced re-cache"
    elif sync_date is None:
        reason = "can't determine last sync date"
    elif date.today() != sync_date:
        reason = "cache age > 0 days"
    if reason:
        print("Downloading issues:", reason)
        issues = _download()
    else:
        issues = cache
    return issues
    
        
skip = {
    3057,  # 3.0 Web FRS
    3066,  # 3.0 Web FRS
    3194,  # URS
}
def relevant(i, sprint_milestone):
    if i.id in skip:
        return False
    if i.sprint_milestone != sprint_milestone:
        return False
    if i.tracker != "Specification":
        return False
    if i.status in ("Rejected", "Closed"):
        return False
    return True

def filter_issues(issues, sm):
    rv = [i for i in issues.values() if relevant(i, sm)]
    print("Filtered %d relevant issues" % len(rv))
    return rv

def get_issues(sm, *, recache=False):
    issues = download(recache)
    return filter_issues(issues, sm)

In [197]:
from scripts.software_frs import frs_traceability2 as FRS

from officelib.xllib import *

xl = Excel(new=False)
ws = xl.Selection.Parent
cells = ws.Cells

In [198]:
_frs_match = re.compile(r"(.*FRS\d+)\.?([\d\.]*)").match
_toplevel_match = re.compile(r"^\>?[\*\+\s]*[\+\*]{2}(FRS[\d\.]+)\:?[\+\*]{2}\:?\s*(.*)$").match
_subitem_match = re.compile(r"^\>?[\+\*]+\s\*(FRS[\d\.]+)\:?\*\:?(.*)$").match
_canceled_match = re.compile(r"^[\+\*]+\s\-\*(FRS[\d\.]+)\:?\*\:?(.*)\-").match
_header_match = re.compile(r"^\>?[\+\*]{2}([\s\d\.\w]+)[\+\*]{2}").match

def _extract_frs_lines(lines):
    start_matches = (
        (_toplevel_match, 0),
        (_subitem_match, 0),
        (_canceled_match, FRS_NA),
    )
    end_matches = (
        _toplevel_match,
        _subitem_match,
        _canceled_match,
        _header_match
    )
    i = 0
    while True:
        try:
            line = lines[i]
        except IndexError:
            break
        i += 1
        for match, flags in start_matches:
            done = False
            m = match(line)
            if m:
                f = m.group(1)
                t = m.group(2).strip()
                l = [t]
                while True:
                    try:
                        line = lines[i]
                    except IndexError:
                        done = True
                        break
                    if any(m(line) for m in end_matches):
                        done = True
                        break
                    l.append(" " + line.strip())
                    i += 1
                text = "\n".join(l)
                # if l and not t:
                #     text = "\n" + text
                yield f, flags, text
            if done:
                break

In [218]:
maybe_frs = re.compile(".*(FRS[\d\.]+)").match
def load_frs_from_issuetracker(sm):
    relevant = get_issues(sm)
    all_frs = {(None, 0, "")}
    seen = set()
    for v in relevant:
        lines = v.description.splitlines()
        found = False
        for frs in _extract_frs_lines(lines):
            all_frs.add(frs)
            seen.add(frs[0])
            found = True
        if not found:
            print("UH OH: ", v.id, v.tracker.name)
        for line in lines:
            m = maybe_frs(line)
            if m:
                if m.group(1) not in seen:
                    print("HMM ", m.group(1))
    all_frs.remove((None, 0, ""))
    return all_frs

In [219]:
def make_paste_data(sm, root):
    data = []
    for i, node in enumerate(root.iter(), 2):
        f = node.id
        t = node.text.strip()
        #if t:
        data.append((sm, f, t))
    return data

In [220]:
ad = [("Version", "Number", "Item")]
smissues = []
for i in range(1, 8):
    sm = "3.0.%d"%i
    smissues.append((sm, load_frs_from_issuetracker(sm)))
    
for sm, frs in smissues:
    reqs = FRS.build_frs_tree(frs)
    ad.extend(make_paste_data(sm, reqs))

Filtered 1 relevant issues
Filtered 1 relevant issues
Filtered 3 relevant issues
UH OH:  3787 Specification
Filtered 3 relevant issues
UH OH:  3825 Specification
Filtered 0 relevant issues
Filtered 1 relevant issues
Filtered 6 relevant issues
UH OH:  3757 Specification
UH OH:  3800 Specification
UH OH:  3865 Specification
UH OH:  3900 Specification
UH OH:  4047 Specification


In [185]:
ws.UsedRange.Clear()
s = cells(2,2)
ds = s.Offset(2, 1)
tr = cells.Range(s, s.Offset(len(ad), len(ad[0])))
dr = cells.Range(ds, ds.Offset(len(ad)-1, len(ad[0])))
tr.Value = ad
tr.Columns.AutoFit()
tr.Rows.AutoFit()

True

In [186]:
for row in dr.Rows:
    row.Cells(1, 2).IndentLevel = row.Cells(1, 2).Value.count(".")